# Query the Dataset in S3 using Amazon Athena and AWS Glue Catalog
This will create an Athena table in the Glue Catalog (Hive Metastore) which allows us to query the dataset files in S3.  We will create a table in the Glue Catalog based on the `Amazon Customer Reviews Dataset` in S3.  We define the columns that map to the data, specify how the data is formatted, and provide the location in Amazon S3 for the file(s).


<img src="img/athena_register_parquet.png" width="60%" align="left">

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=32890294272, available=15909662720, percent=51.6, used=16508215296, free=11085811712, active=17482338304, inactive=3204014080, buffers=0, cached=5296267264, shared=1351680, slab=454778880)


In [3]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

In [4]:
from pyathena import connect
import pandas as pd

In [5]:
%store -r setup_dependencies_passed

In [6]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(setup_dependencies_passed)

True


In [8]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [9]:
%store -r role

# Create Athena Table for Parquet Data

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [10]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [11]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [12]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [13]:
# SQL statement to execute
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date int, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    CREATE EXTERNAL TABLE IF NOT EXISTS default.amazon_reviews_parquet (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date int, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'



""


# Analyze and Build the Partitions

In [14]:
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    MSCK REPAIR TABLE default.amazon_reviews_parquet



""


# Verify The Table Has Been Created Succesfully

In [15]:
statement = "SHOW TABLES in {}".format(database_name)

df_tables = pd.read_sql(statement, conn)
df_tables

,tab_name
0,amazon_reviews_parquet


# Run A Sample Query

In [16]:
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

print(statement)


    SELECT * FROM default.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100



In [17]:
df = pd.read_sql(statement, conn)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,29736021,R3AKKHBV6K3CBH,B00E7XA7KY,189774198,Quickbooks Pro,1,1,1,N,N,Do not buy if you have a MAC and are currently...,for some reason the geniuses over at quickbook...,16171,2014,Digital_Software
1,US,51342536,R10DR245KPSKQ3,B008S0IMCC,534964191,Quicken Deluxe 2013,1,0,0,N,Y,"Wow- Registration does not work, so neither do...",My comment to the Quicken problem report:<br /...,15900,2013,Digital_Software
2,US,3396648,R35TP4JMIUJTYN,B00H9A60O4,608720080,Avast Free Antivirus 2015 [Download],5,0,0,N,N,Avast Review,I have Avast free version and love it . I like...,16171,2014,Digital_Software
3,US,11563861,R1II69Z2AUVA51,B008S0IWNQ,710343641,QuickBooks Pro 2013,1,0,1,N,Y,CAN'T DOWNLOAD,and can't get any assistance and difficult to ...,15900,2013,Digital_Software
4,US,44519372,RAROSQ3C8JR0,B00FFINICE,65804941,TurboTax Deluxe Fed and Efile 2013,5,0,1,N,Y,defacto standard - well designed software,"Quite a racket, this federal income tax, espec...",16171,2014,Digital_Software


In [18]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [19]:
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [20]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Review the New Athena Table in the Glue Catalog

In [21]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

# Store Variables for the Next Notebooks

In [22]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
pipeline_endpoint_name                                -> 'gpt3-model-from-registry-ep-1677492110'
pipeline_experiment_name                              -> 'GPT3-pipeline-1677117939'
pipeline_name                                         -> 'GPT3-pipeline-1677512620'
pipeline_trial_name                                   -> 'trial-1677117943'
role                                                  -> 'arn:aws:iam::079002598131:role/service-role/Amazo
s3_private_path_tsv                                   -> 's3://sagemaker-us-east-1-079002598131/amazon-revi
s3_public_path_tsv                                    -> 's3://amazon-reviews-pds/tsv'
setup_dependencies_passed                             -> True


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>